# Create a User Defined Model using astropy.modeling

## Authors
Rocio Kiman, Lia Corrales and Zé Vinícius.

## Learning Goals
* Know and understand tools to make user defined fitters with `astropy` and in which cases it could be useful
* Learn custom fitters
* This tutorial assumes the student knows how to fit data using `astropy.modeling`. Check the first tutorial in case you are not familiar with it [here](https://github.com/astropy/astropy-tutorials/tree/master/tutorials/notebooks/Models-Quick-Fit).

## Keywords
Modeling, User Defined Fitter, Custom Fitter 

## Summary
In this tutorial, we will learn how to define a new fitter.